# Lecture Plan

1. Pinecone

Org --> Projects --> Indexes


Index Types :

1. serverless
2. pod
   - storage optimized : s1
   - performance optimized: p1, p2



In [5]:
! echo 'PINECONE_API_KEY=your-api-key' >> .env 

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
! pip install pinecone-client

In [7]:
from pinecone import Pinecone

import os

key = os.getenv("PINECONE_API_KEY")

client = Pinecone(api_key=key)

client.describe_index('trng-index')

{'dimension': 1024,
 'host': 'trng-index-i93auiv.svc.gcp-starter.pinecone.io',
 'metric': 'dotproduct',
 'name': 'trng-index',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [ ]:
from pinecone import PodSpec

client.create_index(
    name='starter',
    dimension=1024,
    spec = PodSpec(
        environment='gcp-starter'
    )
)

In [12]:
index = client.Index('trng-index')

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('thenlper/gte-large')


In [14]:
random_sentences = [
    "The sunsets over the horizon, painting the sky in vibrant hues of orange and pink.",
    "Coffee beans release a rich aroma as they grind, signaling the start of a new day.",
    "In the forest, the leaves rustle gently in the breeze, creating a soothing melody.",
    "Technology continues to evolve, shaping the way we communicate and interact with the world !",
    "Lost in a good book, the reader becomes immersed in a world of imagination and adventure.",
    "Laughter echoed through the air as friends gathered around the bonfire on a chilly evening.",
    "The old lighthouse stood tall, its beacon guiding ships safely through the dark night.",
    "Raindrops danced on the window pane, creating a rhythmic melody on a gloomy afternoon.",
    "As the first snowflake fell, the town transformed into a winter wonderland.",
    "A distant train whistle evoked a sense of nostalgia, echoing through the quiet countryside.",
    "Gazing at the stars, one can't help but marvel at the vastness of the universe.",
    "The aroma of freshly baked bread wafted through the bakery, enticing passersby.",
    "Children's laughter filled the playground as they played games and shared stories.",
    "A solitary figure walked along the beach, lost in thought, as waves gently lapped the shore.",
    "The puzzle pieces clicked together, revealing a completed picture and a sense of accomplishment.",
]


In [15]:
embeddings = [model.encode(i) for i in random_sentences]

In [21]:
vectors = []

for i in range(len(random_sentences)):
    vectors.append({'id': str(i), 'values': embeddings[i], 'metadata' : {'text': random_sentences[i]}})


vectors[1]



{'id': '1',
 'values': array([-0.00900913,  0.01461185, -0.01348964, ...,  0.00494261,
        -0.02139146,  0.01867539], dtype=float32),
 'metadata': {'text': 'Coffee beans release a rich aroma as they grind, signaling the start of a new day.'}}

In [22]:
index.upsert(
    vectors= vectors,
    namespace='sample-namespace'

)

{'upserted_count': 15}

In [24]:
query = 'old lighthouse stood tall'

query_vector = model.encode(query)

In [29]:
result = index.query(
    vector=query_vector.tolist(),
    top_k=2,
    namespace='sample-namespace',
    include_values=True,
    include_metadata=True
   
)

result

{'matches': [{'id': '6',
              'metadata': {'text': 'The old lighthouse stood tall, its beacon '
                                   'guiding ships safely through the dark '
                                   'night.'},
              'score': 0.942887604,
              'values': [-0.000176150221,
                         -0.019052377,
                         -0.017799912,
                         0.00358054927,
                         -0.0384365618,
                         0.0171739943,
                         -0.0367441438,
                         0.0295132846,
                         0.048772268,
                         0.00381488027,
                         0.0275578834,
                         -0.00466304598,
                         -0.0293103941,
                         -0.0289831031,
                         0.00133189012,
                         0.0102901235,
                         -0.0415368751,
                         -0.0111733647,
                      

In [ ]:
index.delete(
    ids=['1','2'],
    namespace='sample-namespace'    
)

index.describe_index_stats()